In [3]:
import geopandas as gpd
import geopandas as gpd
import pandas as pd
import numpy as np
from pathlib import Path
from shapely import *

# Importing all the GTFS files

To save on the memory used, we don't import empty columns and use more efficient data types when possible


In [4]:
dir_GTFS= "/home/lubuntu/GSDMA_2024/Tec GTFS"
print("Loading stops")
stops = pd.read_csv(dir_GTFS+"/stops.txt", usecols=['stop_id','stop_name','stop_lat','stop_lon','zone_id','location_type'])
print("Loading stop_times")
stop_times = pd.read_csv(dir_GTFS+"/stop_times.txt", dtype={'service_id':'category',
                                                            'pickup_type':'category',
                                                            'drop_off_type':'category',
                                                            'stop_sequence':'int8',
                                                           'departure_time':'string',
                                                           'arrival_time':'string'}
                        )#,parse_dates=["arrival_time", "departure_time"], date_format="%H:%M")
print("Loading trips")
trips = pd.read_csv(dir_GTFS+"/trips.txt", dtype={'service_id':'category',
                                                  'trip_short_name':'category',
                                                  'direction_id':'int8'})
print("Loading shapes")
shapes = pd.read_csv(dir_GTFS+"/shapes.txt", dtype={'shape_id':'category',
                                                    'shape_pt_sequence':'uint32'})
print("Loading routes")
routes = pd.read_csv(dir_GTFS+"/routes.txt", usecols=['route_id','agency_id','route_short_name','route_long_name','route_type'],
                    dtype={'route_type':'uint8',
                          'agency_id':'category'})
#print("Loading agency")
#agency = pd.read_csv(dir_GTFS+"/agency.txt")
print("Loading calendar")
calendar = pd.read_csv(dir_GTFS+"/calendar.txt",dtype={'monday':'boolean',
                                                      'tuesday':'boolean',
                                                      'wednesday':'boolean',
                                                      'thursday':'boolean',
                                                      'friday':'boolean',
                                                      'saturday':'boolean',
                                                      'sunday':'boolean'},
                      parse_dates=["start_date", "end_date"], date_format="%Y%m%d")
#print("Loading calendar_dates")
#calendar_dates = pd.read_csv(dir_GTFS+"/calendar_dates.txt", dtype={'exception_type':'uint8'}, parse_dates=["date"], date_format="%Y%m%d")

Loading stops
Loading stop_times
Loading trips
Loading shapes
Loading routes
Loading calendar


# Converting the dataframes to geodataframes
## Stops

In [5]:
geometry=gpd.points_from_xy(stops['stop_lon'], stops['stop_lat'], z=None, crs='epsg:4326')
geo_stops=gpd.GeoDataFrame(data=stops, geometry=geometry)
#geo_stops.sample(n=30).plot()
del stops

## Shapes

In [6]:
geometry=gpd.points_from_xy(shapes['shape_pt_lon'], shapes['shape_pt_lat'], z=None, crs='epsg:4326')
geo_shapes=gpd.GeoDataFrame(data=shapes, geometry=geometry)
#geo_shapes.sample(50).plot()
del shapes

In [7]:
#Sort the lines by id and Sequence so they are ordered
geo_shapes_sorted = geo_shapes.sort_values(by=['shape_id', 'shape_pt_sequence'])

#Group the points by 'route'
lines = geo_shapes_sorted.groupby('shape_id', observed=True).apply(lambda x: LineString(x.geometry.tolist()))
lines=lines.reset_index()
lines = gpd.GeoDataFrame(data=lines['shape_id'], geometry=lines[0], crs=geo_shapes.crs)
#lines.sample(20).explore()
del geo_shapes
del geo_shapes_sorted

/tmp/ipykernel_4336/1013893770.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lines = geo_shapes_sorted.groupby('shape_id', observed=True).apply(lambda x: LineString(x.geometry.tolist()))


In [6]:
lines

,shape_id,geometry
0,B00050012,"LINESTRING (4.87717 50.72296, 4.87730 50.72357..."
1,B00050013,"LINESTRING (4.92584 50.80830, 4.92601 50.80819..."
2,B00060086,"LINESTRING (4.61307 50.66783, 4.61302 50.66768..."
3,B00060087,"LINESTRING (4.61307 50.66783, 4.61302 50.66768..."
4,B00060088,"LINESTRING (4.74619 50.79346, 4.74582 50.79333..."
...,...,...
6523,X99800096,"LINESTRING (5.34656 50.23010, 5.34674 50.23013..."
6524,X99800098,"LINESTRING (5.30987 50.18658, 5.30988 50.18658..."
6525,X99800101,"LINESTRING (5.26088 50.32912, 5.26086 50.32919..."
6526,X99800102,"LINESTRING (5.33441 50.36218, 5.33446 50.36225..."


# Direct trips

Find the services corresponding to the given date

In [7]:
departure_date="2024-10-30"
pd.Timestamp(departure_date).weekday()
week_day=pd.Timestamp(departure_date).day_name().lower()
active_services=calendar.query("(start_date<=@departure_date)&(@departure_date<=end_date)")
active_services=active_services[active_services[week_day]]
active_services

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
13,B_2024-BW_A_P2-Mercredi-23,False,False,True,False,False,False,False,2024-10-02,2024-12-11
14,B_2024-BW_A_P2-Mercredi-23-0010000,False,False,True,False,False,False,False,2024-10-02,2024-12-11
31,B_2024-BW_A_P3-Sem-Cong-02,True,True,True,True,True,False,False,2024-10-21,2024-10-31
34,B_2024-BW_A_P3-Sem-Cong-02-0010000,False,False,True,False,False,False,False,2024-10-21,2024-10-31
36,B_2024-BW_A_P3-Sem-Cong-02-0111100,False,True,True,True,True,False,False,2024-10-21,2024-10-31
49,B_2024-BW_P_P2-Semaine-03,True,True,True,True,True,False,False,2024-09-13,2024-12-12
50,B_2024-BW_P_P2-Semaine-03-0010000,False,False,True,False,False,False,False,2024-09-13,2024-12-12
51,B_2024-BW_P_P2-Semaine-03-0111100,False,True,True,True,True,False,False,2024-09-13,2024-12-12
52,B_2024-BW_P_P3-Sem-Cong-02,True,True,True,True,True,False,False,2024-09-27,2024-10-31
53,B_2024-BW_P_P3-Sem-Cong-02-0111100,False,True,True,True,True,False,False,2024-09-27,2024-10-31


## Find the direct trips from one station to the other
For that, find all the trips frome the departure station and the trips to the arrival station.
Then find the trips in common

In [8]:
departure_time="16:00"
departure_stop_name="MONS Place de Flandre"
arrival_stop_name="NIMY Limite"#"SOIGNIES SNCB"

#Find the active services
week_day=pd.Timestamp(departure_date).day_name().lower()
active_services=calendar.query("(start_date<=@departure_date)&(@departure_date<=end_date)")
active_services=active_services[active_services[week_day]]
#Extract the stops and the stop_id's
departure_stop=geo_stops.query("stop_name==@departure_stop_name")
arrival_stop=geo_stops.query("stop_name==@arrival_stop_name")

departure_stop_id=departure_stop['stop_id']
arrival_stop_id=arrival_stop['stop_id']
#Extract the stop times
departure_stop_times=stop_times.query("(stop_id.isin(@departure_stop_id))&(departure_time>@departure_time)")
arrival_stop_times=stop_times.query("stop_id.isin(@arrival_stop_id)&arrival_time>@departure_time")
#Find the trips linking one stop to the other
common_trips = pd.Series(np.intersect1d(departure_stop_times['trip_id'].values,arrival_stop_times['trip_id'].values))
#Filter the active trips
common_active_trips = trips[(trips['trip_id'].isin(common_trips))& (trips['service_id'].isin(active_services['service_id']))]
common_active_trips

final_departure_stop_times=departure_stop_times[departure_stop_times['trip_id'].isin(common_active_trips["trip_id"])].sort_values('departure_time')
final_arrival_stop_times=arrival_stop_times[arrival_stop_times['trip_id'].isin(common_active_trips["trip_id"])].sort_values('arrival_time')

possible_times=pd.merge(final_departure_stop_times,final_arrival_stop_times, on='trip_id', how='inner')
possible_times

,trip_id,arrival_time_x,departure_time_x,stop_id_x,stop_sequence_x,pickup_type_x,drop_off_type_x,arrival_time_y,departure_time_y,stop_id_y,stop_sequence_y,pickup_type_y,drop_off_type_y
0,44639156-H_2024-H24_P3-Sem-Vac-10,16:52:00,16:52:00,H1ms294c,6,0,0,17:08:00,17:08:00,H1ni319a,17,0,0
1,44639685-H_2024-H24_P3-Sem-Vac-10,18:47:00,18:47:00,H1ms294c,6,0,0,19:03:00,19:03:00,H1ni319b,17,0,0
2,44639691-H_2024-H24_P3-Sem-Vac-10,19:27:00,19:27:00,H1ms294c,6,0,0,19:43:00,19:43:00,H1ni319b,17,0,0


In [9]:
best_time=possible_times.iloc[0,:]
ax=departure_stop.explore()
arrival_stop.explore(m=ax)
best_trip=trips[trips['trip_id']==best_time.trip_id]
best_lines=lines[lines['shape_id'].isin(best_trip['shape_id'])]
best_lines.explore(m=ax)

# Trips with transfers

Find the services corresponding to the given date

In [10]:
departure_date="2024-10-30"
pd.Timestamp(departure_date).weekday()
week_day=pd.Timestamp(departure_date).day_name().lower()
active_services=calendar.query("(start_date<=@departure_date)&(@departure_date<=end_date)")
#active_services=active_services[active_services[week_day]]

## Find the trips with one transfer from one station to the other
First naïve method:
Find all the trips passing by the stops and check if there is one in common
If not, list all the stops accessible by these trips and check if there is one in common
If not 


In [11]:
departure_time="10:00"
departure_stop_name="BRAINE-LE-COMTE Gare - Quai 2"
arrival_stop_name="NIVELLES Gare - Quai 3"#"SOIGNIES SNCB"

#Find the active services
week_day=pd.Timestamp(departure_date).day_name().lower()
active_services=calendar.query("(start_date<=@departure_date)&(@departure_date<=end_date)")
active_services=active_services[active_services[week_day]]
#Extract the stops and the stop_id's
departure_stop=geo_stops.query("stop_name==@departure_stop_name")
arrival_stop=geo_stops.query("stop_name==@arrival_stop_name")

departure_stop_id=departure_stop['stop_id']
arrival_stop_id=arrival_stop['stop_id']
#Extract the stop times
departure_stop_times=stop_times.query("(stop_id.isin(@departure_stop_id))&(departure_time>@departure_time)")
arrival_stop_times=stop_times.query("stop_id.isin(@arrival_stop_id)&arrival_time>@departure_time")
#
departure_trip_ids=departure_stop_times['trip_id'].values
arrival_trip_ids=arrival_stop_times['trip_id'].values
#Find the trips linking one stop to the other
common_trips = pd.Series(np.intersect1d(departure_trip_ids, arrival_trip_ids))
#Filter the active trips
common_active_trips = trips[(trips['trip_id'].isin(common_trips))& (trips['service_id'].isin(active_services['service_id']))]
if(common_active_trips.shape[0]<1):
    print("No direct trips")
    print("Searching trips with transfers...")
    stop_times_linked_departure=stop_times.query("(trip_id.isin(@departure_trip_ids))&(departure_time>@departure_time)")
    stop_times_linked_arrival=stop_times.query("(trip_id.isin(@arrival_trip_ids))&(arrival_time>@departure_time)")
    stop_times_linked_departure_ids=stop_times_linked_departure['stop_id'].values
    stop_times_linked_arrival_ids=stop_times_linked_arrival['stop_id'].values
    common_stops = pd.Series(np.intersect1d(stop_times_linked_departure_ids, stop_times_linked_arrival_ids))
    
else:    
    final_departure_stop_times=departure_stop_times[departure_stop_times['trip_id'].isin(common_active_trips["trip_id"])].sort_values('departure_time')
    final_arrival_stop_times=arrival_stop_times[arrival_stop_times['trip_id'].isin(common_active_trips["trip_id"])].sort_values('arrival_time')
    
    possible_times=pd.merge(final_departure_stop_times,final_arrival_stop_times, on='trip_id', how='inner')
    possible_times

## Bruteforcesque method


Some arrival and departure time are above 24 hours (Ex: 24:03:00). They indicate that the bus trip started the previous day and is still considered as active even if it is not today.
To handle those times, the columns 'departure_time' and 'arrival_time' are not stored as datetime but as timedelta.

In [8]:
stop_times['arrival_time']=pd.to_timedelta(stop_times['arrival_time'])
stop_times['departure_time']=pd.to_timedelta(stop_times['departure_time'])

In [22]:
stop_times[stop_times['arrival_time']>pd.to_timedelta('24:00:00')]#7645 stops after midnight

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
466938,41591058-L_PA_2024-24_SP_VA-Sem-Vac-30-0000100,1 days 00:03:00,1 days 00:03:00,LaAbush*,1,0,0
466939,41591058-L_PA_2024-24_SP_VA-Sem-Vac-30-0000100,1 days 00:06:00,1 days 00:06:00,LaAelis2,2,0,0
466940,41591058-L_PA_2024-24_SP_VA-Sem-Vac-30-0000100,1 days 00:07:00,1 days 00:07:00,LaApost2,3,0,0
466941,41591058-L_PA_2024-24_SP_VA-Sem-Vac-30-0000100,1 days 00:09:00,1 days 00:09:00,LaAmise2,4,0,0
466942,41591058-L_PA_2024-24_SP_VA-Sem-Vac-30-0000100,1 days 00:11:00,1 days 00:11:00,LaAhaup2,5,0,0
...,...,...,...,...,...,...,...
5251846,45163517-L_PA_2024-24_LG_DI-Dimanche-11,1 days 00:08:00,1 days 00:08:00,Lheloti1,32,0,0
5251847,45163517-L_PA_2024-24_LG_DI-Dimanche-11,1 days 00:09:00,1 days 00:09:00,Lhehoux1,33,0,0
5251848,45163517-L_PA_2024-24_LG_DI-Dimanche-11,1 days 00:09:00,1 days 00:09:00,Lheneuv1,34,0,0
5251849,45163517-L_PA_2024-24_LG_DI-Dimanche-11,1 days 00:10:00,1 days 00:10:00,Lhecarc1,35,0,0


In [9]:
def compute_walk_time(A_stop, B_stop):
    walk_speed=4#km/h
    #For the moment, I divide the distance between them by 4km/h 
    #In the future, we will use the length of the shortest walkable path between the points 

    #The stops are projected in the Belgian Lambert 2008 coordinates system (crs=3812) to have accurate distances
    if(type(B_stop)==gpd.geodataframe.GeoDataFrame):
        B_stop=B_stop.head(1)
        A_stop=A_stop.head(1)
    birdfly_dist=A_stop.distance(B_stop, align=False)
    walk_time=birdfly_dist/(1000*walk_speed/60)#walking time in minutes (float)
    #walk_time=walk_time.mean()#If several stops have the same name, take the average of their walking distance
    walk_time=pd.to_timedelta(walk_time,unit='min')#Converted to TimeDelta
    return walk_time

In [46]:
active_stop_times[active_stop_times['stop_id'].isin(departure_stop_id)]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type


In [40]:
def find_next_stops(best_arrival_time, active_trips, active_stop_times, cur_stop_id, cur_time):
    walk_range=1000#Max length in meters between the current stop and the other stops reached by foot
    
    #walk_time=compute_walk_time(arrival_stop, cur_stop)
    #Extract trips stopping by the current bus stop that are active
    #print(cur_time)
    cur_stop_times=active_stop_times[((active_stop_times['stop_id'].isin(cur_stop_id))&
                                     (active_stop_times['departure_time']>cur_time)&
                                     (active_stop_times['departure_time']<(best_arrival_time))
                                     #(active_stop_times[trip_id].isin(active_trips_id))
                                      )].sort_values('departure_time')
#    cur_stop_times=active_stop_times.query("(stop_id.isin(@cur_stop_id))"
#                                    +"&(departure_time>@cur_time)"
#                                    #+"&(trip_id.isin(@active_trips_id))"
#                                     ).sort_values('departure_time')
    
    #Extract the other stops that can be reached with the trips
    other_stops_times=[]
    for row in cur_stop_times.itertuples(index=False):
        departure_seq=int(row.stop_sequence)
        other_stops_time=active_stop_times[((active_stop_times['trip_id']==row.trip_id)
                                      &(active_stop_times['arrival_time']<(cur_time+walk_time))
                                      &(active_stop_times['stop_sequence']>departure_seq)
                                     )]
        other_stops_times.append(other_stops_time)
        #active_stop_times=active_stop_times[~active_stop_times.eq(other_stops_time,axis=0).all(axis=1)]
        #print(row.departure_time, end='\r')

    #Add the stops reachable by foot
    cur_stop=geo_stops[geo_stops['stop_id'].isin(cur_stop_id)]
    meas_geo_stops=geo_stops.to_crs(3812)
    walked_stops=meas_geo_stops[meas_geo_stops.within(cur_stop.to_crs(3812).buffer(1000).geometry.iloc[0])]
    
    if(len(other_stops_times)>0):
        other_stops_times=pd.concat(other_stops_times)
    else:
        print("No next stops found")
        return pd.DataFrame()
    return other_stops_times
def explore_node(geo_stops, active_trips, active_stop_times, found_stop_times, best_arrival_time, best_path, cur_stop_time):
    num_trans=cur_stop_time['number_trips']+1
    #print(cur_stop_time[['stop_id','final_arrival_time', 'intermediary_stops']])
    num_trans=num_trans.iloc[0]
    max_trans=6
    if(num_trans>max_trans):
        print("Max number of transfers reached")
        return pd.DataFrame(), best_arrival_time, best_path
    cur_stop_id=cur_stop_time['stop_id']#pd.Series(row.stop_id)
    cur_time=cur_stop_time['arrival_time'].iloc[0]#row.arrival_time
    previous_stops=cur_stop_time['intermediary_stops'].iloc[0].copy()#row.intermediary_stops[:]
    #Extract the stops directly linked to the current stop
    #print(previous_stops)
    other_stops_times=find_next_stops(best_arrival_time, active_trips, active_stop_times, cur_stop_id, cur_time)

    if(other_stops_times.size>0):
        #Update the path going to these stops
        previous_stops.append(cur_stop_id.iloc[0])
        #print(previous_stops)
        other_stops_times['number_trips']=num_trans
        other_stops_times['intermediary_stops']=[previous_stops] * len(other_stops_times)#TODO: also include the trips used (or walking)
        
        #Compute the proximity of the stops discovered from the arrival stop
        other_stops=geo_stops.merge(other_stops_times.reset_index(), on='stop_id',how='right', )
        new_walk_times=compute_walk_time(other_stops.to_crs(3812), arrival_stop.to_crs(3812).iloc[0].geometry)
        other_stops['final_arrival_time']=other_stops['arrival_time']+new_walk_times
        other_stops=other_stops.set_index('index')
        new_best_arrival_time=other_stops['final_arrival_time'].min()
        if(new_best_arrival_time<best_arrival_time):
            #Update the current best path
            best_path=[previous_stops, 'w']
            best_arrival_time=new_best_arrival_time

            
        other_stops_times['final_arrival_time']=other_stops['final_arrival_time']
    else:
        return pd.DataFrame(), best_arrival_time, best_path
    return other_stops_times, best_arrival_time, best_path

In [11]:
geo_stops[geo_stops['stop_name'].str.contains('Kennedy')].iloc[0:10]

,stop_id,stop_name,stop_lat,stop_lon,zone_id,location_type,geometry
2383,Brixaug1,RIXENSART Rue de l'Augette (Av. Kennedy),50.707132,4.521891,6248,0,POINT (4.52189 50.70713)
2384,Brixaug2,RIXENSART Rue de l'Augette (Av. Kennedy),50.707066,4.521723,6248,0,POINT (4.52172 50.70707)
2414,Brixpro3,RIXENSART Route Provinciale (Av. Kennedy),50.711275,4.520952,6248,0,POINT (4.52095 50.71128)
7839,H1ms913a,MONS Bd JF Kennedy,50.460221,3.959520,3101,0,POINT (3.95952 50.46022)
14747,LHdkenn1,HODY Rond-point Kennedy,50.486268,5.504035,5242,0,POINT (5.50404 50.48627)
14748,LHdkenn2,HODY Rond-point Kennedy,50.486682,5.503668,5242,0,POINT (5.50367 50.48668)
14749,LHdkenn3,HODY Rond-point Kennedy,50.486770,5.503248,5242,0,POINT (5.50325 50.48677)
21344,N138aja,GONRIEUX Résidence Kennedy,50.046323,4.429460,4138,0,POINT (4.42946 50.04632)
21345,N138ajb,GONRIEUX Résidence Kennedy,50.045924,4.429358,4138,0,POINT (4.42936 50.04592)


In [59]:
departure_date="2024-11-05"
departure_time="08:00:00"
departure_stop_name="MONS Place de Flandre"#"MONS Lycée"
arrival_stop_name="SOIGNIES Place du Jeu de Balle"

#Extract the stops and the stop_id's
departure_stop=geo_stops.query("stop_name==@departure_stop_name")
arrival_stop=geo_stops.query("stop_name==@arrival_stop_name")

arrival_stop_id=arrival_stop['stop_id']
departure_stop_id=departure_stop['stop_id']

#The walking time between the departure and the arrival is computed
walk_time=compute_walk_time(departure_stop.to_crs(3812), arrival_stop.to_crs(3812))
walk_time=walk_time.mean()

departure_time=pd.to_timedelta(departure_time)
best_arrival_time=departure_time+walk_time
print(f"First arrival time: {best_arrival_time}")
best_path=['w']

#Find the schedule of the day
week_day=pd.Timestamp(departure_date).day_name().lower()
active_services=calendar.query("(start_date<=@departure_date)&(@departure_date<=end_date)")
active_services=active_services[active_services[week_day]]
active_trips=trips[trips['service_id'].isin(active_services['service_id'])]
active_trips_id=active_trips['trip_id']
active_stop_times=stop_times[stop_times['trip_id'].isin(active_trips_id)]
active_stop_times=active_stop_times[active_stop_times['arrival_time']<best_arrival_time]

#Find the stoptimes of the departure stop
found_stop_times=active_stop_times[((active_stop_times['stop_id'].isin(departure_stop_id))&
                                     (active_stop_times['departure_time']>departure_time)&
                                     (active_stop_times['departure_time']<(best_arrival_time))
                                     #(active_stop_times[trip_id].isin(active_trips_id))
                                      )].sort_values('departure_time')
#TODO: Add other stops within walkable distance

#print(found_stop_times)
found_stop_times['number_trips']=0
found_stop_times.loc[:, "intermediary_stops"] = np.array([['']] * len(found_stop_times))
found_stop_times['final_arrival_time']=best_arrival_time

other_stops_times=find_next_stops(best_arrival_time, active_trips, active_stop_times, departure_stop_id, departure_time)

found_stop_times['explored']=True
other_stops_times['number_trips']=1
other_stops_times.loc[:, "intermediary_stops"] = [[departure_stop_id.iloc[0]]] * len(other_stops_times) #other_stops_times['intermediary_stops']=''
#TODO: Add other stops within walkable distance
other_stops_times=other_stops_times.sort_values('arrival_time').drop_duplicates('stop_id')#We keep only the earliest arrival time to a stop.
other_stops_times['explored']=False

#Compare the walking time from the new stops
other_stops=geo_stops.merge(other_stops_times.reset_index(), on='stop_id',how='right', )
new_walk_times=compute_walk_time(other_stops.to_crs(3812), arrival_stop.to_crs(3812).iloc[0].geometry)
other_stops['final_arrival_time']=other_stops['arrival_time']+new_walk_times
other_stops=other_stops.set_index('index')
new_best_arrival_time=other_stops['final_arrival_time'].min()
if(new_best_arrival_time<best_arrival_time):
    #TODO: Update the current best path
    best_path=[departure_stop_id.iloc[0], 'w']
    best_arrival_time=new_best_arrival_time
    #Drop the scheduled stops later than the current best arrival time 
    active_stop_times=active_stop_times[active_stop_times['arrival_time']<best_arrival_time]

other_stops_times['final_arrival_time']=other_stops['final_arrival_time']
found_stop_times=pd.concat([found_stop_times,other_stops_times])
found_stop_times=found_stop_times.sort_values('arrival_time').drop_duplicates('stop_id')#We keep only the earliest arrival time to a stop.
found_stop_times=found_stop_times.sort_values('final_arrival_time')

new_stop_times=found_stop_times[found_stop_times['explored']==False]#The found stops not already explored


while(new_stop_times.size>0):
    new_stop_times=found_stop_times[found_stop_times['explored']==False]#The found stops not already explored
    #TODO: New Structure
    cur_stop_time=new_stop_times.iloc[0:1,:]

    other_stops_times, new_best_arrival_time, new_best_path=explore_node(geo_stops, active_trips, active_stop_times, found_stop_times, best_arrival_time, best_path, cur_stop_time)
    other_stops_times['explored']=False
    found_stop_times.loc[cur_stop_time.index,'explored']=True

    #Drop the scheduled stops later than the current best arrival time
    if(new_best_arrival_time<best_arrival_time):
        print(f"New best arrival time: {new_best_arrival_time}")
        best_arrival_time=new_best_arrival_time
        best_path=new_best_path
        active_stop_times=active_stop_times[active_stop_times['arrival_time']<best_arrival_time]
        found_stop_times=found_stop_times[found_stop_times['arrival_time']<best_arrival_time]

    found_stop_times=pd.concat([found_stop_times,other_stops_times])
    found_stop_times=found_stop_times.sort_values('arrival_time')
    found_stop_times=found_stop_times.drop_duplicates('stop_id')#We keep only the earliest arrival time to a stop.
    found_stop_times=found_stop_times.sort_values('final_arrival_time')
    new_stop_times=found_stop_times[found_stop_times['explored']==False]#The found stops not already explored
    print(f"{found_stop_times.shape[0]-new_stop_times.shape[0]}/{found_stop_times.shape[0]}")
print(f"Meilleur temps:{best_arrival_time}")
print(f"Meilleur chemin:{best_path}")

First arrival time: 0 days 12:02:09.688000704
3/248
4/248
5/248
6/248
7/248
8/248
9/248
10/248
11/248
12/248
13/248
14/248
15/248
16/248
17/248
18/248
19/248
20/248
21/248
22/248
23/248
24/248
25/248
26/248
14/248
15/248
16/248
17/248
18/248
19/248
20/248
21/248
22/248
23/248
24/248
25/248
26/248
27/248
28/248
29/248
30/248
31/248
32/248
21/248
22/248
23/248
24/248
25/248
26/248
27/248
28/248
29/248
30/248
31/248
32/248
33/248
34/248
35/248
36/248
36/248
37/248
38/248
39/248
40/248
32/248
33/248
34/248
35/248
36/248
37/248
38/248
39/248
40/248
41/248
42/248
43/248
44/248
42/248
43/248
44/248
45/248
46/248
47/248
48/248
46/248
47/248
48/248
49/248
50/248
51/248
49/248
50/248
51/248
52/248
50/248
51/248
52/248
53/248
41/248
42/248
43/248
44/248
45/248
46/248
47/248
48/248
49/248
50/248
51/248
52/248
53/248
54/248
55/248
56/248
57/248
58/248
58/248
59/248
60/248
54/248
55/248
56/248
57/248
58/248
59/248
60/248
61/248
62/248
58/248
59/248
60/248
61/248
62/248
63/248
62/248
63/248
64/248
65

In [46]:
best_arrival_time

Timedelta('0 days 11:44:27.433295826')

In [54]:
found_stop_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,number_trips,intermediary_stops,final_arrival_time,explored
5338194,45184776-H_2024-H24_P2-Sem-N-3-11-0100000,0 days 09:33:00,0 days 09:33:00,H3so175a,46.0,0,0,1.0,[H1ms294a],0 days 09:33:00,True
5338193,45184776-H_2024-H24_P2-Sem-N-3-11-0100000,0 days 09:31:00,0 days 09:31:00,H3so155a,45.0,0,0,3.0,"[H1ms294a, H1ms304a, H1ms267a]",0 days 09:36:31.072196268,True
5338195,45184776-H_2024-H24_P2-Sem-N-3-11-0100000,0 days 09:34:00,0 days 09:34:00,H3so170a,47.0,0,0,1.0,[H1ms294a],0 days 09:41:23.439053448,True
5338192,45184776-H_2024-H24_P2-Sem-N-3-11-0100000,0 days 09:30:00,0 days 09:30:00,H3so166d,44.0,0,0,3.0,"[H1ms294a, H1ms304a, H1ni323a]",0 days 09:42:00.645615372,True
5338191,45184776-H_2024-H24_P2-Sem-N-3-11-0100000,0 days 09:30:00,0 days 09:30:00,H3so166a,43.0,0,0,4.0,"[H1ms294a, H1ms401a, H1ms276a, H1ms253a]",0 days 09:42:07.902825354,True
...,...,...,...,...,...,...,...,...,...,...,...
5357461,45185124-H_2024-H24_P2-Sem-N-3-11,0 days 09:26:00,0 days 09:26:00,H1al107a,42.0,0,0,4.0,"[H1ms294a, H1ms401a, H1ms276a, H1ms252d]",0 days 16:39:53.812449186,True
5357464,45185124-H_2024-H24_P2-Sem-N-3-11,0 days 09:28:00,0 days 09:28:00,H1al106a,45.0,0,0,3.0,"[H1ms294a, H1ms401a, H1hn208a]",0 days 16:41:37.073152764,True
5357463,45185124-H_2024-H24_P2-Sem-N-3-11,0 days 09:28:00,0 days 09:28:00,H1al108a,44.0,0,0,3.0,"[H1ms294a, H1ms401a, H1hn208a]",0 days 16:44:49.080212718,True
5357462,45185124-H_2024-H24_P2-Sem-N-3-11,0 days 09:27:00,0 days 09:27:00,H1al105a,43.0,0,0,3.0,"[H1ms294a, H1ms401a, H1hn208a]",0 days 16:46:07.103824386,True


In [60]:
found_stop_times['departure_time']=found_stop_times['departure_time'].apply(lambda x: x.total_seconds())
found_stop_times['final_arrival_time']=found_stop_times['final_arrival_time'].apply(lambda x: x.total_seconds())
found_stop_times['arrival_time']=found_stop_times['arrival_time'].apply(lambda x: x.total_seconds())

In [61]:
geo_stops.merge(found_stop_times, how='right').sort_values('final_arrival_time').explore(column='final_arrival_time')

In [39]:
best_path

['w']